In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('df_2021_LIMPIO.csv')

/var/folders/r7/9s3q_3s92lz959y8hxyn782c0000gn/T/ipykernel_1381/2878934528.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_2021_LIMPIO.csv')


In [5]:
df.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro,pagare,capital,fechaAperturaCredito,idBancoStr,idRespuestaBancoStr
0,101188,599055,27240538,2,1765.53,1765.53,0.0,NaN,04,07/10/2022,183615.12,70000.00,2021-08-13,NaN,Cuenta Insuficiencia Fondos
1,100551,579674,27163668,21,506.38,506.38,0.0,NaN,DD00031,29/09/2022,48612.48,18204.19,2021-02-09,NaN,Cuenta con Insuficiencia de Fondos
2,100551,581020,27163709,21,4899.99,4899.99,0.0,NaN,DD00031,29/09/2022,470399.04,176153.00,2021-02-23,NaN,Cuenta con Insuficiencia de Fondos
3,100551,581980,27163662,127,444.82,444.82,0.0,NaN,DD00031,29/09/2022,16013.52,10000.00,2021-03-06,NaN,Cuenta con Insuficiencia de Fondos
4,100551,582409,27163723,21,847.85,847.85,0.0,NaN,DD00031,29/09/2022,81393.60,30480.00,2021-03-11,NaN,Cuenta con Insuficiencia de Fondos


In [11]:
# Hacer un csv con las primeras 100 filas del dataframe
df.head(100).to_csv('df_2021_LIMPIO_100.csv', index=False)

In [6]:
df.isnull().sum()


idListaCobro                  0
idCredito                     0
consecutivoCobro              0
idBanco                       0
montoExigible                 0
montoCobrar                   0
montoCobrado                  0
fechaCobroBanco         6504510
idRespuestaBanco             49
fechaEnvioCobro               0
pagare                        0
capital                       0
fechaAperturaCredito          0
idBancoStr              6823855
idRespuestaBancoStr         657
dtype: int64

In [7]:
import pandas as pd

# Cargar tu DataFrame principal
df['fechaCobroBanco'] = pd.to_datetime(df['fechaCobroBanco'], errors='coerce')
df['exitoso'] = df['montoCobrado'] > 0

# Features por crédito
features = df.groupby('idCredito').agg(
    n_intentos=('idListaCobro', 'count'),
    n_exitosos=('exitoso', 'sum'),
    n_fallidos=('exitoso', lambda x: (~x).sum()),
    porcentaje_exito=('exitoso', 'mean'),
    dias_desde_ultimo_pago=('fechaCobroBanco', lambda x: (pd.Timestamp.today() - x.max()).days if x.max() else None),
    max_monto_cobrado=('montoCobrado', 'max'),
    avg_monto_cobrado=('montoCobrado', 'mean'),
).reset_index()

# Detectar si falló los últimos 2 intentos
fallos_ultimos = df.sort_values(['idCredito', 'fechaCobroBanco']).groupby('idCredito')['exitoso'].tail(2).groupby(df['idCredito']).apply(lambda x: (x == False).sum() == 2)
features['fallo_2_ultimos'] = features['idCredito'].map(fallos_ultimos).fillna(False).astype(int)



features.head()


/var/folders/r7/9s3q_3s92lz959y8hxyn782c0000gn/T/ipykernel_1381/3623529883.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fechaCobroBanco'] = pd.to_datetime(df['fechaCobroBanco'], errors='coerce')


,idCredito,n_intentos,n_exitosos,n_fallidos,porcentaje_exito,dias_desde_ultimo_pago,max_monto_cobrado,avg_monto_cobrado,fallo_2_ultimos
0,577278,1,0,1,0.0,NaN,0.0,0.0,0
1,577279,1,0,1,0.0,NaN,0.0,0.0,0
2,577281,2,1,1,0.5,1157.0,1843.0,921.5,0
3,577284,39,0,39,0.0,NaN,0.0,0.0,1
4,577285,2,0,2,0.0,NaN,0.0,0.0,1


In [8]:
features.to_csv('features_idcredito.csv', index=False)

In [9]:
def clasificar_estrategia(row):
    if row['n_fallidos'] == 0:
        return 'gratis'
    elif row['n_fallidos'] == 1:
        return 'inmediato'
    elif row['fallo_2_ultimos'] == 1:
        return 'despriorizado'
    elif row['porcentaje_exito'] >= 0.8 and row['n_intentos'] >= 5:
        return 'gratis'
    else:
        return 'inmediato'

# Aplica la función a cada fila
features['estrategia_sugerida'] = features.apply(clasificar_estrategia, axis=1)

# Verifica algunas filas
print(features[['idCredito', 'n_intentos', 'n_fallidos', 'fallo_2_ultimos', 'porcentaje_exito', 'estrategia_sugerida']].head())

# Opcional: guardar como CSV
# features.to_csv("features_entrenamiento_con_estrategia.csv", index=False)


   idCredito  n_intentos  n_fallidos  fallo_2_ultimos  porcentaje_exito  \
0     577278           1           1                0               0.0   
1     577279           1           1                0               0.0   
2     577281           2           1                0               0.5   
3     577284          39          39                1               0.0   
4     577285           2           2                1               0.0   

  estrategia_sugerida  
0           inmediato  
1           inmediato  
2           inmediato  
3       despriorizado  
4       despriorizado  


In [10]:
features.head()

,idCredito,n_intentos,n_exitosos,n_fallidos,porcentaje_exito,dias_desde_ultimo_pago,max_monto_cobrado,avg_monto_cobrado,fallo_2_ultimos,estrategia_sugerida
0,577278,1,0,1,0.0,NaN,0.0,0.0,0,inmediato
1,577279,1,0,1,0.0,NaN,0.0,0.0,0,inmediato
2,577281,2,1,1,0.5,1157.0,1843.0,921.5,0,inmediato
3,577284,39,0,39,0.0,NaN,0.0,0.0,1,despriorizado
4,577285,2,0,2,0.0,NaN,0.0,0.0,1,despriorizado


In [12]:
features.to_csv("features_entrenamiento_con_estrategia.csv", index=False)

In [13]:
import pandas as pd

# Carga tu dataset original
df = pd.read_csv('features_entrenamiento_con_estrategia.csv')

# Define estrategias disponibles y sus características
estrategias_info = {
    'confiable':      {'costo': 0, 'tiempo': 6},
    'tentativo':      {'costo': 6, 'tiempo': 0.5},
    'desconfiable':   {'costo': 3, 'tiempo': 2},
    'stand-by':       {'costo': 0, 'tiempo': 6}
}

# Expandir el DataFrame: una fila por crédito y estrategia
expanded_rows = []
for _, row in df.iterrows():
    for estrategia, config in estrategias_info.items():
        # Determinar si la estrategia es válida según las reglas de degradación
        fallidos = row['n_fallidos']
        if estrategia == 'confiable' and fallidos == 0:
            permitida = 1
        elif estrategia == 'tentativo' and fallidos == 1:
            permitida = 1
        elif estrategia == 'desconfiable' and fallidos == 2:
            permitida = 1
        elif estrategia == 'stand-by' and fallidos >= 3:
            permitida = 1
        else:
            permitida = 0

        expanded_rows.append({
            'idCredito': row['idCredito'],
            'estrategia': estrategia,
            'costo_comision': config['costo'],
            'tiempo_respuesta': config['tiempo'],
            'permitida': permitida,
            'n_fallidos': row['n_fallidos'],
            'porcentaje_exito': row['porcentaje_exito'],
            'avg_monto_cobrado': row['avg_monto_cobrado'],
            'probabilidad_exito': row['porcentaje_exito'],  # para ILP futuro
            'valor_esperado': row['porcentaje_exito'] * row['avg_monto_cobrado']
        })

# Crear el DataFrame expandido
df_exp = pd.DataFrame(expanded_rows)

# Guardar o mostrar
df_exp.to_csv('df_expandidos_ilp.csv', index=False)
print("✅ Dataset expandido guardado como df_expandidos_ilp.csv")


✅ Dataset expandido guardado como df_expandidos_ilp.csv


In [14]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD

# Cargar el dataset expandido
df_exp = pd.read_csv('df_expandidos_ilp.csv')

# Parámetro de penalización para tiempo de respuesta
lambda_tiempo = 1  # puedes ajustar este valor

# Crear el modelo
model = LpProblem("Asignación_Estrategias_Cobranza", LpMinimize)

# Crear variables binarias: x_{i,j} = 1 si se asigna estrategia j al crédito i
decision_vars = {}
for _, row in df_exp.iterrows():
    key = (row['idCredito'], row['estrategia'])
    decision_vars[key] = LpVariable(f"x_{int(row['idCredito'])}_{row['estrategia']}", 0, 1, LpBinary)

# Función objetivo: minimizar costo + penalización por tiempo
model += lpSum(
    (row['costo_comision'] + lambda_tiempo * row['tiempo_respuesta']) * decision_vars[(row['idCredito'], row['estrategia'])]
    for _, row in df_exp[df_exp['permitida'] == 1].iterrows()
)

# Restricción: solo una estrategia por crédito (y debe ser válida)
for id_credito in df_exp['idCredito'].unique():
    estrategias_validas = df_exp[(df_exp['idCredito'] == id_credito) & (df_exp['permitida'] == 1)]
    model += lpSum(decision_vars[(id_credito, row['estrategia'])] for _, row in estrategias_validas.iterrows()) == 1

# Resolver el modelo
solver = PULP_CBC_CMD(msg=True)
model.solve(solver)

# Extraer resultados
asignaciones = []
for (id_credito, estrategia), var in decision_vars.items():
    if var.varValue == 1:
        asignaciones.append({'idCredito': id_credito, 'estrategia_asignada': estrategia})

df_resultado = pd.DataFrame(asignaciones)
df_resultado.to_csv('asignacion_optima.csv', index=False)
print("✅ Asignaciones óptimas guardadas en 'asignacion_optima.csv'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/josecanto/Library/Python/3.9/lib/python/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/r7/9s3q_3s92lz959y8hxyn782c0000gn/T/dd6727a429864d34a80aca76ec10fbd9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/r7/9s3q_3s92lz959y8hxyn782c0000gn/T/dd6727a429864d34a80aca76ec10fbd9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 51159 COLUMNS
At line 255776 RHS
At line 306931 BOUNDS
At line 358086 ENDATA
Problem MODEL has 51154 rows, 51154 columns and 51154 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 306718 - 0.02 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 306718 to -1.79769e+308
Probing was tried

In [15]:
df_resultado.head()

,idCredito,estrategia_asignada
0,577278,tentativo
1,577279,tentativo
2,577281,tentativo
3,577284,stand-by
4,577285,desconfiable


In [26]:
import pandas as pd

# Cargar los datos
df_original = pd.read_csv("features_entrenamiento_con_estrategia.csv")
df_resultado = pd.read_csv("asignacion_optima.csv")

# Hacer merge sincronizado por idCredito
df_final = df_original.merge(df_resultado, on="idCredito", how="left")

# Verificar merge (opcional)
assert df_final.shape[0] == df_original.shape[0], "⚠️ El merge desalineó el número de filas."

df_final.drop(columns=['estrategia_sugerida'], inplace=True)

# Guardar resultado
df_final.to_csv("features_con_estrategia_optima.csv", index=False)
print("✅ Merge exitoso. Estrategia asignada añadida al DataFrame.")


✅ Merge exitoso. Estrategia asignada añadida al DataFrame.


In [22]:
import pandas as pd

# Cargar los datos
df_original = pd.read_csv("df_2021_LIMPIO.csv")
df_resultado = pd.read_csv("asignacion_optima.csv")

# Hacer merge sincronizado por idCredito
df_final = df_original.merge(df_resultado, on="idCredito", how="left")

# Verificar merge (opcional)
assert df_final.shape[0] == df_original.shape[0], "⚠️ El merge desalineó el número de filas."

# Guardar resultado
df_final.to_csv("features_con_estrategia_optima_2021.csv", index=False)
print("✅ Merge exitoso. Estrategia asignada añadida al DataFrame.")

/var/folders/r7/9s3q_3s92lz959y8hxyn782c0000gn/T/ipykernel_1381/250615868.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv("df_2021_LIMPIO.csv")


✅ Merge exitoso. Estrategia asignada añadida al DataFrame.


In [27]:
df_credito = pd.read_csv("features_con_estrategia_optima.csv")

In [28]:
df_credito.head()

,idCredito,n_intentos,n_exitosos,n_fallidos,porcentaje_exito,dias_desde_ultimo_pago,max_monto_cobrado,avg_monto_cobrado,fallo_2_ultimos,estrategia_asignada
0,577278,1,0,1,0.0,NaN,0.0,0.0,0,tentativo
1,577279,1,0,1,0.0,NaN,0.0,0.0,0,tentativo
2,577281,2,1,1,0.5,1157.0,1843.0,921.5,0,tentativo
3,577284,39,0,39,0.0,NaN,0.0,0.0,1,stand-by
4,577285,2,0,2,0.0,NaN,0.0,0.0,1,desconfiable


In [29]:
df_2021 = pd.read_csv("features_con_estrategia_optima_2021.csv")

/var/folders/r7/9s3q_3s92lz959y8hxyn782c0000gn/T/ipykernel_1381/1816583031.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv("features_con_estrategia_optima_2021.csv")


In [30]:
df_2021.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaEnvioCobro,pagare,capital,fechaAperturaCredito,idBancoStr,idRespuestaBancoStr,estrategia_asignada
0,101188,599055,27240538,2,1765.53,1765.53,0.0,NaN,04,07/10/2022,183615.12,70000.00,2021-08-13,NaN,Cuenta Insuficiencia Fondos,stand-by
1,100551,579674,27163668,21,506.38,506.38,0.0,NaN,DD00031,29/09/2022,48612.48,18204.19,2021-02-09,NaN,Cuenta con Insuficiencia de Fondos,stand-by
2,100551,581020,27163709,21,4899.99,4899.99,0.0,NaN,DD00031,29/09/2022,470399.04,176153.00,2021-02-23,NaN,Cuenta con Insuficiencia de Fondos,stand-by
3,100551,581980,27163662,127,444.82,444.82,0.0,NaN,DD00031,29/09/2022,16013.52,10000.00,2021-03-06,NaN,Cuenta con Insuficiencia de Fondos,stand-by
4,100551,582409,27163723,21,847.85,847.85,0.0,NaN,DD00031,29/09/2022,81393.60,30480.00,2021-03-11,NaN,Cuenta con Insuficiencia de Fondos,stand-by
